In [ ]:
import numpy as np
import math
import heapq
import queue
import gurobi
import time
import itertools
import functools
import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline

np.random.seed(1)
np.set_printoptions(precision=4)

In [ ]:
isPlotModel = False
ifPrintPlan = False
runOur_oneStage = False
runOur_twoStage = False
runOur_oneStage_random = False
runOur_twoStage_random = False
runRoyset2 = False
runRoyset3 = False
runDp = False
runBrute = False

## Generate Model

In [ ]:
def generateModel_sparse(agentN, locN, stageN, sparseProb=0.6, isPrintModel=True):
    # Generate the prior distribution vector
    priorVec = np.random.random(locN)
    priorVec /= priorVec.sum()
    if isPrintModel:
        print('The prior probability vector is:\n' + str(priorVec) + '\n')

    # Generate the Markov transition probability matrix (left stochastic matrix; each column sums up to one)
    tranProbMat = np.random.rand(locN, locN)
    tranProbMat /= tranProbMat.sum(axis=0)[None,:]
    if isPrintModel:
        print('The Markov transition probability matrix (x=Px) is:\n' + str(tranProbMat) + '\n')

    # Generate the probability vector of successful detections (between 0.5 to 1.0)
    detectVec = 1 - 0.5 * np.random.random(locN)
    if isPrintModel:
        print('The probability vector of successful detections is:\n' + str(detectVec) + '\n')

    # Compute the probability vector of missed detections
    missVec = 1 - detectVec
    if isPrintModel:
        print('The probability vector of failed detections is:\n' + str(missVec) + '\n')
        
    # Generate access pairs
    accessSource = dict([(m, []) for m in range(agentN)])  # key=source, value=sinks_from_this_source
    accessSink = dict([(k, []) for k in range(locN)])  # key=sink, value=sources_to_this_sink
    for m in range(agentN):
        for k in range(locN):
            if np.random.binomial(1, sparseProb) == 1:
                accessSource[m].append(k)
                accessSink[k].append(m)
        # If agent m cannot access any location, add one
        if len(accessSource[m]) == 0:
            k = np.random.randint(0, locN)
            accessSource[m].append(k)
            accessSink[k].append(m)
    for k in range(locN):
        # If location k cannot be accessed any agent, add one
        if len(accessSink[k]) == 0:
            m = np.random.randint(0, agentN)
            accessSource[m].append(k)
            accessSink[k].append(m)
    if isPrintModel:
        print(accessSource)
        print(accessSink)
    
    # Return model
    return [priorVec, tranProbMat, detectVec, missVec, accessSource, accessSink]

In [ ]:
# agentN - number of agents
agentN = 5
# locN - number of locations
locN = 81
# stageN - number of stages
stageN = 8

[priorVec, tranProbMat, detectVec, missVec, accessSource, accessSink] = \
    generateModel_sparse(agentN, locN, stageN, sparseProb=1, isPrintModel=False)

## Plot Model

In [ ]:
def plot(agentN, locN, accessSource, accessSink):
    G = nx.MultiDiGraph()
    # Add sources
    for m in range(agentN):
        G.add_node('s' + str(m))
    # Add sinks
    for k in range(agentN):
        G.add_node('t' + str(k))
    # Add arcs
    for m in range(agentN):
        for k in accessSource[m]:
            G.add_edge('s' + str(m), 't' + str(k))
    # Configure node positions
    nodes_pos_dict = {}
    for i in range(agentN):
        nodes_pos_dict['s'+str(i)] = (0, - (i - (agentN - 1) / 2))
    for i in range(locN):
        nodes_pos_dict['t'+str(i)] = (locN / 2, - (i - (locN - 1) / 2))
    nodes_pos_dict['gt'] = (locN, 0)
    # Configure plot dimensions
    plt.rcParams['figure.figsize'] = (10, 6)
    # Plot model
    nx.draw_networkx(G, pos = nodes_pos_dict)

In [ ]:
if isPlotModel:
    plot(agentN, locN, accessSource, accessSink)

## Utility Functions

In [ ]:
def computeObjVal(stageN, priorVec, tranProbMat, missVec, actionVec):
    vec = np.copy(priorVec)
    for s in range(stageN):
        vec = tranProbMat.dot(np.multiply(np.power(missVec, actionVec[s]), vec))
    return sum(vec)

## Subroutines for Solving Subproblems

In [ ]:
def solveRelaxedSubproblem_sparse(agentN, locN, missVec, q, accessSource, accessSink):
    action = [ 0 ] * locN
    baction = [ [ 0 ] * locN for m in range(agentN) ]
    prevObjVal = sum(q)
    
    agent_id = 0
    itr = 1
    while True:
        lamb = 0
        for k in accessSource[agent_id]:
            lamb = max(lamb, -q[k] * math.log(missVec[k]))
        left = 0.0
        right = lamb
        while True:
            # Solve for b_m_k with current lamb
            for k in accessSource[agent_id]:
                if (q[k] * math.log(missVec[k]) + lamb >= 0):
                    baction[agent_id][k] = 0
                else:
                    tmp = -lamb / (q[k] * math.log(missVec[k]))
                    baction[agent_id][k] = math.log(tmp, missVec[k])
                    for m in range(agentN):
                        if m != agent_id:
                            baction[agent_id][k] -= baction[m][k]
            should_be_one = sum(baction[agent_id])
            if math.fabs(should_be_one - 1) <= 1e-14:
                break
            elif should_be_one > 1: # Increase lamb
                tmp = lamb
                lamb = (lamb + right) / 2.0
                left = tmp
            else: # Decrease lamb
                tmp = lamb
                lamb = (left + lamb) / 2.0
                right = tmp
            itr += 1
        for k in range(locN):
            action[k] = 0
            for m in range(agentN):
                action[k] += baction[m][k]
        objVal = sum([q[k] * math.pow(missVec[k], action[k]) for k in range(locN)])
        if objVal >= prevObjVal:
            break
        prevObjVal = objVal
        agent_id = (agent_id + 1) % agentN
    return action

In [ ]:
def assign_extra_demand(target_sink, agentN, locN, accessSource, accessSink, baction, supply, eliminated):
    # Initialization
    visited = [ False ] * (agentN + locN)
    Q = queue.Queue()
    Q.put(target_sink+agentN)
    in_Q = [ False ] * (agentN + locN)
    in_Q[target_sink+agentN] = True
    pred = {} # Record predecessors for building augmenting path
    for nd in range(agentN + locN):
        pred[nd] = None
    
    # Main loop
    while Q.empty() is False:
        nd = Q.get()
        in_Q[nd] = False
        visited[nd] = True
        # If the node is a sink
        if nd >= agentN:
            for m in accessSink[nd-agentN]:
                if in_Q[m] == False and visited[m] == False and eliminated[m] == False:
                    Q.put(m)
                    in_Q[m] = True
                    pred[m] = nd
        # If the node is a source, but it has no unused supply
        elif nd < agentN and supply[nd] == 0:
            for k in accessSource[nd]:
                if baction[nd][k] > 0 and in_Q[k+agentN] == False \
                        and visited[k+agentN] == False and eliminated[k+agentN] == False:
                    Q.put(k+agentN)
                    in_Q[k+agentN] = True
                    pred[k+agentN] = nd
        # If the node is a source, and it has unused supply
        else:
            # Decrement unused_supply of the source by 1
            assert(supply[nd] == 1)
            supply[nd] -= 1
            # Recursively build augmenting path
            cur = nd
            while True:
                pre = pred[cur]
                if pre == None:
                    break
                # For source
                if cur < agentN:
                    baction[cur][pre-agentN] += 1
                else:
                    baction[pre][cur-agentN] -= 1
                cur = pre
            # There is no node to be eliminated! And remember to break the outer loop!
            visited = []
            break
    return visited

def solveIntegerSubproblem_sparse(agentN, locN, detectVec, missVec, q, accessSource, accessSink):
    # Initialize "b"-action (agent-wise & location-wise action)
    baction = [ [0] * locN for m in range(agentN) ]
    
    # Initialize heap
    heap = []
    for k in range(locN):
        heapq.heappush(heap, (- q[k] * detectVec[k], k))
    
    # Initilize supply
    supply = [ 1 ] * agentN
    total_supply = agentN
    
    # Initilize sets of eliminated nodes
    eliminated = [ False ] * (agentN + locN)
    
    # Main loop
    while total_supply > 0:
        ele = heapq.heappop(heap)
        k = ele[1]
        if eliminated[k+agentN]:
            continue
        toBeEliminated = assign_extra_demand(k, agentN, locN, accessSource, accessSink, baction, supply, eliminated)
        if len(toBeEliminated) == 0:
            heapq.heappush(heap, (ele[0] * missVec[k], k)) # insert new weight into heap
            total_supply -= 1
        else:
            for nd in range(agentN + locN):
                if toBeEliminated[nd]:
                    eliminated[nd] = True
    
    # Compute location-wise action
    action = np.sum(baction, axis=0)
    return action

## Forward-Backward Iterative Algorithm

In [ ]:
def forBackIter_sparse(agentN, locN, stageN, priorVec, tranProbMat, detectVec, missVec, accessSource, accessSink, relaxedRecur=True): 
    # Initialize the decision variables (indexes of locations chosen at each stage)
    # Alert: if use [[0]*locN]*stageN, then we have stageN lists referencing to the same list
    actionVec = [ [0] * locN for s in range(stageN) ] 
    
    # Initialize qR
    qR = np.empty([stageN,locN])
    qR[0,:] = priorVec

    # Initialize qL
    qL = np.ones((stageN,locN))
    
    # Relaxed Forward-Backward Recursions
    if relaxedRecur == True:
        prevObjVal = 1.0
        cycleN = 0
        while True:
            cycleN += 1
            # Forward recursion
            print('\n==== Relaxed forward recursion starts ====\n')
            for s in range(stageN-1): # s=0,...,T-2
                print('\n---Stage ' + str(s) + '---\n')
                # Solve subproblem
                q = [ a*b for a,b in zip(qL[s,:],qR[s,:]) ]
                actionVec[s] = solveRelaxedSubproblem_sparse(agentN, locN, missVec, q, \
                                                             accessSource, accessSink)
                # Update qR
                tmp_q = np.copy(qR[s,:])  # must use np.copy, otherwise they reference the same array
                tmp_Is = np.power(missVec, actionVec[s])
                qR[s+1,:] = tranProbMat.dot(np.multiply(tmp_Is, tmp_q))
            # Backward recursion
            print('\n==== Relaxed backward recursion starts ====\n')
            for s in reversed(range(1,stageN)): # s=T-1,...,1
                print('\n---Stage ' + str(s) + '---\n')
                # Solve subproblem
                q = [ a*b for a,b in zip(qL[s,:],qR[s,:]) ]
                actionVec[s] = solveRelaxedSubproblem_sparse(agentN, locN, missVec, q, \
                                                             accessSource, accessSink)                # Update qL
                tmp_q = np.copy(qL[s,:])  # must use np.copy, otherwise they reference the same array
                tmp_Is = np.power(missVec, actionVec[s])
                qL[s-1,:] = tranProbMat.T.dot(np.multiply(tmp_Is, tmp_q)) # remember to transpose the transition matrix

            # Compute new objective value
            curObjVal = computeObjVal(stageN, priorVec, tranProbMat, missVec, actionVec)
            print('\nThe objective value becomes: %.4f' % (curObjVal) + '\n')
            # Decide if the relaxed recursions terminates
            if curObjVal >= prevObjVal:
                break
            prevObjVal = curObjVal
    
    # Integer Forward-Backward Recursions
    prevObjVal = 1.0
    cycleN = 0
    isPrintAction = False
    while True:
        cycleN += 1
        # Forward recursion
        print('\n==== Integer forward recursion starts ====\n')
        for s in range(stageN-1): # s=0,...,T-2
            print('\n---Stage ' + str(s) + '---\n')
            # Solve subproblem
            q = [ a*b for a,b in zip(qL[s,:],qR[s,:]) ]
            actionVec[s] = solveIntegerSubproblem_sparse(agentN, locN, detectVec, missVec, q, \
                                                         accessSource, accessSink)
            if isPrintAction:
                print('\nThe actions become:\n')
                print(actionVec)
            # Update qR
            tmp_q = np.copy(qR[s,:])  # must use np.copy, otherwise they reference the same array
            tmp_Is = np.power(missVec, actionVec[s])
            qR[s+1,:] = tranProbMat.dot(np.multiply(tmp_Is, tmp_q))
        # Backward recursion
        print('\n==== Integer backward recursion starts ====\n')
        for s in reversed(range(1,stageN)): # s=T-1,...,1
            print('\n---Stage ' + str(s) + '---\n')
            # Solve subproblem
            q = [ a*b for a,b in zip(qL[s,:],qR[s,:]) ]
            actionVec[s] = solveIntegerSubproblem_sparse(agentN, locN, detectVec, missVec, q, \
                                                         accessSource, accessSink)
            if isPrintAction:
                print('\nThe actions become:\n')
                print(actionVec)
            # Update qL
            tmp_q = np.copy(qL[s,:])  # must use np.copy, otherwise they reference the same array
            tmp_Is = np.power(missVec, actionVec[s])
            qL[s-1,:] = tranProbMat.T.dot(np.multiply(tmp_Is, tmp_q)) # remember to transpose the transition matrix
        
        # Compute new objective value
        curObjVal = computeObjVal(stageN, priorVec, tranProbMat, missVec, actionVec)
        print('\nThe objective value becomes: %.4f' % (curObjVal) + '\n')
        # Decide if the algorithm terminates (must run more than one cycle)
        if curObjVal >= prevObjVal and cycleN > 1:
            return [curObjVal, actionVec, cycleN]
        prevObjVal = curObjVal
    return [curObjVal, actionVec, cycleN]

In [ ]:
if runOur_oneStage:
    start_time_our_oneStage = time.time()
    [optObjVal_our_oneStage, optActionVec_our_oneStage, cycleN_our_oneStage] = \
        forBackIter_sparse(agentN, locN, stageN, priorVec, tranProbMat, \
                           detectVec, missVec, accessSource, accessSink, \
                           relaxedRecur=False)
    time_elapsed_our_oneStage = time.time() - start_time_our_oneStage

In [ ]:
if runOur_twoStage:
    start_time_our_twoStage = time.time()
    [optObjVal_our_twoStage, optActionVec_our_twoStage, cycleN_our_twoStage] = \
        forBackIter_sparse(agentN, locN, stageN, priorVec, tranProbMat, \
                           detectVec, missVec, accessSource, accessSink, \
                           relaxedRecur=True)
    time_elapsed_our_twoStage = time.time() - start_time_our_twoStage

## Forward-Backward Iterative Algorithm (Random)

In [ ]:
def forBackIterRandom_sparse(agentN, locN, stageN, priorVec, tranProbMat, detectVec, missVec, accessSource, accessSink, relaxedRecur=True, randomRepetitions=10): 
    # Initialize the decision variables (indexes of locations chosen at each stage)
    # Alert: if use [[0]*locN]*stageN, then we have stageN lists referencing to the same list
    actionVec = [ [0] * locN for s in range(stageN) ] 

    # Initialize qR
    qR = np.empty([stageN,locN])
    qR[0,:] = priorVec

    # Initialize qL
    qL = np.ones((stageN,locN))
    
    # Relaxed Forward-Backward Recursions
    if relaxedRecur == True:
        prevObjVal = 1.0
        cycleN = 0
        isPrintQ = False
        while True:
            cycleN += 1
            # Forward recursion
            print('\n==== Relaxed forward recursion starts ====\n')
            for s in range(stageN-1): # s=0,...,T-2
                #print('\n---Stage ' + str(s) + '---\n')
                # Solve subproblem
                q = [ a*b for a,b in zip(qL[s,:],qR[s,:]) ]
                actionVec[s] = solveIntegerSubproblem_sparse(agentN, locN, detectVec, missVec, q, \
                                                             accessSource, accessSink)
                # Update qR
                tmp_q = np.copy(qR[s,:])  # must use np.copy, otherwise they reference the same array
                tmp_Is = np.power(missVec, actionVec[s])
                qR[s+1,:] = tranProbMat.dot(np.multiply(tmp_Is, tmp_q))
            # Backward recursion
            print('\n==== Relaxed backward recursion starts ====\n')
            for s in reversed(range(1,stageN)): # s=T-1,...,1
                #print('\n---Stage ' + str(s) + '---\n')
                # Solve subproblem
                q = [ a*b for a,b in zip(qL[s,:],qR[s,:]) ]
                actionVec[s] = solveIntegerSubproblem_sparse(agentN, locN, detectVec, missVec, q, \
                                                             accessSource, accessSink)
                # Update qL
                tmp_q = np.copy(qL[s,:])  # must use np.copy, otherwise they reference the same array
                tmp_Is = np.power(missVec, actionVec[s])
                qL[s-1,:] = tranProbMat.T.dot(np.multiply(tmp_Is, tmp_q)) # remember to transpose the transition matrix

            # Compute new objective value
            curObjVal = computeObjVal(stageN, priorVec, tranProbMat, missVec, actionVec)
            print('\nThe objective value becomes: %.4f' % (curObjVal) + '\n')
            # Decide if the relaxed recursions terminates
            if curObjVal >= prevObjVal:
                break
            prevObjVal = curObjVal
        actionDistVec = np.array(actionVec)
        row_sums = actionDistVec.sum(axis=1)
        actionDistVec = actionDistVec / row_sums[:, np.newaxis]
    else:
        actionDistVec = [ [1.0 / locN] * locN ] * stageN
    
    optObjVal = 1.0
    for rep in range(randomRepetitions):
        # Randomly generate intial actionVec
        actionVec = [ [0] * locN ] * stageN
        for s in range(stageN):
            actionVec[s] = np.random.multinomial(agentN, actionDistVec[s])

        # Compute qL (no need to compute qR here)
        for s in reversed(range(1, stageN)):
            tmp_q = np.copy(qL[s,:])
            tmp_Is = np.power(missVec, actionVec[s])
            qL[s-1,:] = tranProbMat.T.dot(np.multiply(tmp_Is, tmp_q))
        
        # Integer Forward-Backward Recursions
        prevObjVal = 1.0
        cycleN = 0
        isPrintAction = False
        while True:
            cycleN += 1
            # Forward recursion
            print('\n==== Rep ' + str(rep) + ': Integer forward recursion starts ====\n')
            for s in range(stageN-1): # s=0,...,T-2
                print('\n---Stage ' + str(s) + '---\n')
                # Solve subproblem
                q = [ a*b for a,b in zip(qL[s,:],qR[s,:]) ]
                actionVec[s] = solveIntegerSubproblem_sparse(agentN, locN, detectVec, missVec, q, \
                                                             accessSource, accessSink)
                if isPrintAction:
                    print('\nThe actions become:\n')
                    print(actionVec)
                # Update qR
                tmp_q = np.copy(qR[s,:])  # must use np.copy, otherwise they reference the same array
                tmp_Is = np.power(missVec, actionVec[s])
                qR[s+1,:] = tranProbMat.dot(np.multiply(tmp_Is, tmp_q))
            # Backward recursion
            print('\n==== Rep ' + str(rep) + ': Integer backward recursion starts ====\n')
            for s in reversed(range(1,stageN)): # s=T-1,...,1
                print('\n---Stage ' + str(s) + '---\n')
                # Solve subproblem
                q = [ a*b for a,b in zip(qL[s,:],qR[s,:]) ]
                actionVec[s] = solveIntegerSubproblem_sparse(agentN, locN, detectVec, missVec, q, \
                                                             accessSource, accessSink)
                if isPrintAction:
                    print('\nThe actions become:\n')
                    print(actionVec)
                # Update qL
                tmp_q = np.copy(qL[s,:])  # must use np.copy, otherwise they reference the same array
                tmp_Is = np.power(missVec, actionVec[s])
                qL[s-1,:] = tranProbMat.T.dot(np.multiply(tmp_Is, tmp_q)) # remember to transpose the transition matrix

            # Compute new objective value
            curObjVal = computeObjVal(stageN, priorVec, tranProbMat, missVec, actionVec)
            print('\nThe objective value becomes: %.4f' % (curObjVal) + '\n')
            # Decide if the algorithm terminates (must run more than one cycle)
            if curObjVal >= prevObjVal and cycleN > 1:
                break
            prevObjVal = curObjVal
        if curObjVal < optObjVal:
            optObjVal = curObjVal
            optActionVec = actionVec
    return [optObjVal, optActionVec, randomRepetitions]

In [ ]:
if runOur_oneStage_random:
    start_time_our_oneStage_random = time.time()
    [optObjVal_our_oneStage_random, optActionVec_our_oneStage_random, randomRepetitionN_our_oneStage_random] = \
        forBackIterRandom_sparse(agentN, locN, stageN, priorVec, tranProbMat, \
                                 detectVec, missVec, accessSource, accessSink, \
                                 relaxedRecur=False, randomRepetitions=20)
    time_elapsed_our_oneStage_random = time.time() - start_time_our_oneStage_random

In [ ]:
if runOur_twoStage_random:
    start_time_our_twoStage_random = time.time()
    [optObjVal_our_twoStage_random, optActionVec_our_twoStage_random, randomRepetitionN_our_twoStage_random] = \
        forBackIterRandom_sparse(agentN, locN, stageN, priorVec, tranProbMat, \
                                 detectVec, missVec, accessSource, accessSink, \
                                 relaxedRecur=True, randomRepetitions=20)
    time_elapsed_our_twoStage_random = time.time() - start_time_our_twoStage_random

## Cutting-Plane Algorithm 2

### In paper "Route Optimization for Multiple Searchers" by Royset and Sato

In [ ]:
def roysetSatoAlg2_sparse(agentN, locN, stageN, priorVec, tranProbMat, detectVec, missVec, accessSource, accessSink, delta=1e-10):
    # Build Gurobi model
    model = gurobi.Model("Royset-Sato Alg 2")
    
    # Add variables - b
    b = {}
    for s in range(stageN):
        for m in range(agentN):
            for k in accessSource[m]:
                b[s,m,k] = model.addVar(obj=0, vtype='B', lb=0, ub=1, name='b'+str(s)+','+str(m)+','+str(k))
    # Add variable - xi
    xi = model.addVar(obj=1.0, name='xi')
    
    # Add constraints: sum_by_loc(b) = 1 for each agent
    for s in range(stageN):
        for m in range(agentN):
            coef = [ 1 for k in accessSource[m] ]
            var = [ b[s,m,k] for k in accessSource[m] ]
            model.addConstr(gurobi.LinExpr(coef,var), '=', 1, name='con_b'+str(s)+'_'+str(m))
    
    # Do not print Gurobi messages
    model.setParam('OutputFlag', False)
    # Update model
    model.update()

    # Cutting-plane
    total_time = 0.0  # total time used
    start_time = time.time()  # start time
    hi = 1.0  # upper bound of optimal value
    lo = 0.0  # lower bound of optimal value
    prev_b = [ [ [0] * locN for m in range(agentN) ] for s in range(stageN) ]
    delta_i = 0.0  # expected MIP optimality gap for problem P_i
    delta_i_half_flag = False  # if lower bound has not changed after one iteration, halve delta_i
    itr = 1  # iteration counter
    while True:
        # Compute current value and update upper bound of optimal value
        prev_a = np.sum(prev_b, axis=1)
        f_a = computeObjVal(stageN, priorVec, tranProbMat, missVec, prev_a)
        if f_a < hi:
            hi = f_a
        
        # Decide if terminates
        if hi - lo <= delta * lo:
            optActionVec = prev_a
            break
        
        # Compute qR for prev_a
        qR = np.empty([stageN,locN])
        qR[0,:] = priorVec
        for s in range(stageN-1): # s=0,...,T-2
            tmp_q = np.copy(qR[s,:])  # must use np.copy, otherwise they reference the same array
            tmp_Is = np.power(missVec, prev_a[s])
            qR[s+1,:] = tranProbMat.dot(np.multiply(tmp_Is, tmp_q))
        
        # Compute qL for prev_a
        qL = np.ones((stageN,locN))
        for s in reversed(range(1,stageN)): # s=T-1,...,1
            tmp_q = np.copy(qL[s,:])  # must use np.copy, otherwise they reference the same array
            tmp_Is = np.power(missVec, prev_a[s])
            qL[s-1,:] = tranProbMat.T.dot(np.multiply(tmp_Is, tmp_q)) # remember to transpose the transition matrix
        
        # Add one more constraint
        constr_lhs = f_a
        for s in range(stageN):
            for k in range(locN):
                prev_a_s_k = 0
                a_s_k = 0
                for m in accessSink[k]:
                    prev_a_s_k += prev_b[s][m][k]
                    a_s_k += b[s,m,k]
                constr_lhs += qL[s][k] * qR[s][k] * (missVec[k] ** prev_a_s_k) \
                              * (-detectVec[k]) * (a_s_k - prev_a_s_k)
        model.addConstr(constr_lhs, '<=', xi, name='cut'+str(itr))
        
        # Set MIP optimality gap = (Upper - Lower) / Upper
        if itr == 2:
            delta_i = min(0.03, (hi - lo) / lo)
        elif itr > 2:
            if delta_i < 1e-10:
                delta_i = 0.0
            else:
                if delta_i_half_flag:
                    delta_i /= 2.0
                delta_i = min(delta_i, 0.03)
                delta_i = min(delta_i, (hi - lo) / lo)
        model.setParam("MIPGap", delta_i / (1 + delta_i));
        
        # Update model and solve
        model.update()
        model.write('gurobi.lp')
        model.optimize()
        
        # Retrieve Gurobi outputs
        for s in range(stageN):
            for m in range(agentN):
                for k in accessSource[m]:
                    prev_b[s][m][k] = model.getVarByName('b'+str(s)+','+str(m)+','+str(k)).X
        MIP_solution = model.getVarByName('xi').X
        if MIP_solution > lo:
            lo = MIP_solution
            delta_i_half_flag = False
        else:
            delta_i_half_flag = True  # if lower bound does not change, halve delta_i in next iteration
        
        # Update total time
        total_time = time.time() - start_time
        
        # Compute relative optimality gap:
        if lo != 0.0:
            g_i = (hi - lo) / lo
        else:
            g_i = math.inf
        
        # Print debugging info
        print("iter=%s, rela_gap=%.5f, hi=%.4e, lo=%.4e, fun_value=%.4e, MIP_solution=%.4e, \
               delta_i=%.3e, time_taken=%.1f" \
               % (itr, g_i, hi, lo, f_a, MIP_solution, \
                  delta_i, total_time))
        
        # Decide if terminates
        if hi - lo <= delta * lo:
            optActionVec = prev_a
            break
        
        itr += 1
        if total_time > 900:
            break
    optObjVal = computeObjVal(stageN, priorVec, tranProbMat, missVec, optActionVec)
    return [optObjVal, optActionVec]

In [ ]:
if runRoyset2:
    start_time_royset2 = time.time()
    [optObjVal_royset2, optActionVec_royset2] = \
        roysetSatoAlg2_sparse(agentN, locN, stageN, priorVec, tranProbMat, \
                              detectVec, missVec, accessSource, accessSink, \
                              delta=1e-10)
    time_elapsed_royset2 = time.time() - start_time_royset2

## Cutting-Plane Algorithm 3

### In paper "Route Optimization for Multiple Searchers" by Royset and Sato

### Alert: Some bugs exist!

In [ ]:
def roysetSatoAlg3_sparse(agentN, locN, stageN, priorVec, tranProbMat, detectVec, missVec, accessSource, accessSink, delta=1e-10):
    # Build Gurobi model
    model = gurobi.Model("Royset-Sato Alg 3")
    
    # Add variables - a
    a = {}
    for s in range(stageN):
        for k in range(locN):
            a[s,k] = model.addVar(obj=0, vtype='C', lb=0, ub=agentN, name='a'+str(s)+','+str(k))  # set a as continuous and 0<=a(s,k)<=M
    # Add variables - b
    b = {}
    for s in range(stageN):
        for m in range(agentN):
            for k in accessSource[m]:
                b[s,m,k] = model.addVar(obj=0, vtype='C', lb=0, ub=1, name='b'+str(s)+','+str(m)+','+str(k))
    # Add variable - xi
    xi = model.addVar(obj=1.0, name='xi')
    
    # Add constraints: sum_by_agent(b) = a for each loc
    for s in range(stageN):
        for k in range(locN):
            coef = [ 1 for m in accessSink[k] ]
            var = [ b[s,m,k] for m in accessSink[k] ]
            model.addConstr(gurobi.LinExpr(coef,var), '=', a[s,k])
    # Add constraints: sum_by_loc(b) = 1 for each agent
    for s in range(stageN):
        for m in range(agentN):
            coef = [ 1 for k in accessSource[m] ]
            var = [ b[s,m,k] for k in accessSource[m] ]
            model.addConstr(gurobi.LinExpr(coef,var), '=', 1)
    
    # Do not print Gurobi messages
    model.setParam('OutputFlag', False)
    # Update model
    model.update()

    # Cutting-plane
    total_time = 0.0  # total time used
    start_time = time.time()  # start time
    hi = 1.0  # upper bound of optimal value
    lo = 0.0  # lower bound of optimal value
    prev_a = [ [0] * locN for s in range(stageN) ]  # initialize T-period actions
    delta_i = 0.0  # expected MIP optimality gap for problem P_i
    delta_i_half_flag = False  # if lower bound has not changed after one iteration, halve delta_i
    hasSwap_relax_to_int_flag = False  # if has just swapped from continuous relaxtion to integer problem
                                       # if not done yet, need to do a.setAttr('vtype', 'I')
    itr = 1  # iteration counter
    while True:
        # Compute current value and update upper bound of optimal value
        f_a = computeObjVal(stageN, priorVec, tranProbMat, missVec, prev_a)
        if f_a < hi:
            hi = f_a
            optActionVec = prev_a
        
        # Decide if terminates
        if hi - lo <= delta * lo:
            optActionVec = prev_a
            break
        
        # Compute relative optimality gap:
        if lo != 0.0:
            g_i = (hi - lo) / lo
        else:
            g_i = math.inf
        
        # Compute qR for prev_a
        qR = np.empty([stageN,locN])
        qR[0,:] = priorVec
        for s in range(stageN-1): # s=0,...,T-2
            tmp_q = np.copy(qR[s,:])  # must use np.copy, otherwise they reference the same array
            tmp_Is = np.power(missVec, prev_a[s])
            qR[s+1,:] = tranProbMat.dot(np.multiply(tmp_Is, tmp_q))

        # Compute qL for prev_a
        qL = np.ones((stageN,locN))
        for s in reversed(range(1,stageN)): # s=T-1,...,1
            tmp_q = np.copy(qL[s,:])  # must use np.copy, otherwise they reference the same array
            tmp_Is = np.power(missVec, prev_a[s])
            qL[s-1,:] = tranProbMat.T.dot(np.multiply(tmp_Is, tmp_q)) # remember to transpose the transition matrix

        # Add one more constraint
        constr_lhs = f_a
        if g_i > 1e-3 and total_time < 600:  # solve continuous relaxation first
            for s in range(stageN):
                for k in range(locN):
                    constr_lhs += qL[s][k] * qR[s][k] * (missVec[k] ** prev_a[s][k]) \
                                  * math.log(missVec[k]) * (a[s,k] - prev_a[s][k])
        else:
            # If this is the first time swapping from relaxed to integer, reset variable attribute
            if not hasSwap_relax_to_int_flag:
                for s in range(stageN):
                    for k in range(locN):
                        a[s,k].setAttr('vtype', 'I')
                for s in range(stageN):
                    for m in range(agentN):
                        for k in accessSource[m]:
                            b[s,m,k].setAttr('vtype', 'I')
                print("iter=%s, rela_gap=%.6f, hi=%s, lo=%s, delta_i=%s, time_taken=%.2f" \
                      % (itr, g_i, hi, lo, delta_i, total_time))
                print("========== Swapping from relaxed to integer ==========")
                hasSwap_relax_to_int_flag = True
                itr = 1
            for s in range(stageN):
                for k in range(locN):
                    constr_lhs += qL[s][k] * qR[s][k] * (missVec[k] ** prev_a[s][k]) \
                                  * (-detectVec[k]) * (a[s,k] - prev_a[s][k])
        model.addConstr(constr_lhs, '<=', xi)
        
        # Set Gurobi MIP optimality gap
        if itr == 2:
            delta_i = min(0.03, g_i / 3)
        elif itr > 2:
            if delta_i_half_flag:
                delta_i /= 2.0
            delta_i = min(delta_i, 0.03)
            delta_i = min(delta_i, g_i / 3)
        gurobiMipGap = delta_i / (1 + delta_i)  # Gurobi_optimality_gap = (hi-lo)/hi, thus need conversion
        model.setParam("MIPGap", gurobiMipGap);
        
        # Update and solve model
        model.update()
        model.optimize()
        
        # Retrieve Gurobi outputs
        prev_a = [ [0] * locN for s in range(stageN) ]
        for s in range(stageN):
            for k in range(locN):
                prev_a[s][k] = model.getVarByName('a'+str(s)+','+str(k)).X
        MIP_solution = model.getVarByName('xi').X
        if MIP_solution > lo:
            lo = MIP_solution
            delta_i_half_flag = False
        else:
            delta_i_half_flag = True  # if lower bound does not change, halve delta_i in next iteration
        
        # Update total time
        total_time = time.time() - start_time
        
        # Compute relative optimality gap:
        if lo != 0.0:
            g_i = (hi - lo) / lo
        else:
            g_i = math.inf
        
        # Print debugging info
        if not hasSwap_relax_to_int_flag:
            if itr % 50 == 1:
                print("iter=%s, rela_gap=%.6f, hi=%.10f, lo=%.10f, delta_i=%.4e, time_taken=%.1f" \
                      % (itr, g_i, hi, lo, delta_i, total_time))
        else:
            print("iter=%s, rela_gap=%.6f, hi=%.10f, lo=%.10f, delta_i=%.4e, time_taken=%.1f" \
                  % (itr, g_i, hi, lo, delta_i, total_time))
        
        # Decide if terminates
        if hi - lo <= delta * lo:
            optActionVec = prev_a
            break
        
        itr += 1
        if total_time > 900:
            break
    optObjVal = computeObjVal(stageN, priorVec, tranProbMat, missVec, optActionVec)
    return [optObjVal, optActionVec]

In [ ]:
if runRoyset3:
    start_time_royset3 = time.time()
    [optObjVal_royset3, optActionVec_royset3] = \
        roysetSatoAlg3_sparse(agentN, locN, stageN, priorVec, tranProbMat, \
                              detectVec, missVec, accessSource, accessSink, \
                              delta=1e-10)
    time_elapsed_royset3 = time.time() - start_time_royset3

## POMDP (Dynamic Programming)

### In paper "The Optimal Search for a Moving Target When the Search Path Is Constrained" by James N. Eagle

In [ ]:
def eaglePomdp_sparse(agentN, locN, stageN, priorVec, tranProbMat, detectVec, missVec, accessSource, accessSink, isPrune=True): 
    actionVecSet= [[]]
    alphaVecSet = []
    alphaVecSet.append(np.zeros(locN))
    
    possibleActions = []
    for locId in itertools.product(*[ li for li in accessSource.values() ]):
        action = [ 0 ] * locN
        for k in locId:
            action[k] += 1
        possibleActions.append(action)
    
    isPrintMiddleSteps = False
    for s in reversed(range(stageN)):
        print('\n---Stage ' + str(s) + '---\n')
        newActionVecSet = []
        newAlphaVecSet = []
        for i, alphaVec in enumerate(alphaVecSet):
            for action in possibleActions:
                # update alpha vectors
                tmpAlphaVec = np.copy(alphaVec)
                identityMat = np.identity(locN)
                for k in range(locN):
                    identityMat[k, k] = missVec[k] ** action[k]
                newAlphaVec = identityMat.dot(tranProbMat.T).dot(tmpAlphaVec)
                extraVec = np.zeros(locN)
                for k in range(locN):
                    if action[k] > 0:
                        extraVec[k] = detectVec[k] ** action[k]
                newAlphaVec += extraVec
                newAlphaVecSet.append(newAlphaVec)
                # update associated actions
                newActionVec = list(actionVecSet[i])
                newActionVec.insert(0, action)
                newActionVecSet.append(newActionVec)        
        actionVecSet = newActionVecSet
        alphaVecSet = newAlphaVecSet
        # Prune dominated alpha vectors
        if isPrune:
            # Get indices of alpha vectors to be pruned
            pruneIndices = []
            for i in range(len(alphaVecSet)):
                alpha_i = alphaVecSet[i]
                for j in range(len(alphaVecSet)):
                    if j == i:
                        continue
                    alpha_j = alphaVecSet[j]
                    if functools.reduce(lambda x, y: x*y, np.less(alpha_i, alpha_j)):
                        pruneIndices.append(i)
                        break
            # Prune the dominated alpha vectors
            for i in sorted(pruneIndices, reverse=True):
                del alphaVecSet[i]
                del actionVecSet[i]
        if isPrintMiddleSteps:
            print('\nThe alpha vectors are:\n')
            print(alphaVecSet)
            print('\nThe actions are:\n')
            print(actionVecSet)
            print('\n')

    optObjVal = 0.0
    for i, alphaVec in enumerate(alphaVecSet):
        tmpObjVal = sum(np.multiply(alphaVec, priorVec))
        if tmpObjVal > optObjVal:
            optObjVal = tmpObjVal
            optActions = actionVecSet[i]
    optObjVal = 1 - optObjVal
    return [optObjVal, optActions]

In [ ]:
if runDp:
    start_time_dp = time.time()
    [optObjVal_dp, optActions_dp] = \
        eaglePomdp_sparse(agentN, locN, stageN, priorVec, tranProbMat, \
                          detectVec, missVec, accessSource, accessSink, isPrune=True)
    time_elapsed_dp = time.time() - start_time_dp

## Brute Force

In [ ]:
def bruteForce_sparse(agentN, locN, stageN, priorVec, tranProbMat, detectVec, missVec, accessSource, accessSink):
    optObjVal = 1.0
    possibleActions = []
    for locId in itertools.product(*[ li for li in accessSource.values() ]):
        action = [ 0 ] * locN
        for k in locId:
            action[k] += 1
        possibleActions.append(action)
    
    for actionVec in itertools.product(possibleActions, repeat=stageN):
        tmp = computeObjVal(stageN, priorVec, tranProbMat, missVec, actionVec)
        if tmp < optObjVal:
            optObjVal = tmp
            optActionVec = actionVec
    return [optObjVal, optActionVec]

In [ ]:
if runBrute:
    start_time_brute = time.time()
    [optObjVal_brute, optActionVec_brute] = \
        bruteForce_sparse(agentN, locN, stageN, priorVec, tranProbMat, \
                          detectVec, missVec, accessSource, accessSink)
    time_elapsed_brute = time.time() - start_time_brute

## Final Results

In [ ]:
if runOur_oneStage:
    print('\n=====For-Back Iter (With One Stage) Final Results=====\n')
    print('The optimal objective value (prob of miss detection) is ' + str(optObjVal_our_oneStage) + ' .\n')
    if ifPrintPlan:
        print('\nThe optimal search plan is:\n\n' + str(optActionVec_our_oneStage) + '\n')
    print('Time taken to run is %.2f' % (time_elapsed_our_oneStage) + ' .\n')
    print('The algorithm converges in ' + str(cycleN_our_oneStage) + ' cycles.\n')
    
if runOur_twoStage:
    print('\n=====For-Back Iter (With Two Stages) Final Results=====\n')
    print('The optimal objective value (prob of miss detection) is ' + str(optObjVal_our_twoStage) + ' .\n')
    if ifPrintPlan:
        print('\nThe optimal search plan is:\n\n' + str(optActionVec_our_twoStage) + '\n')
    print('Time taken to run is %.2f' % (time_elapsed_our_twoStage) + ' .\n')
    print('The algorithm converges in ' + str(cycleN_our_twoStage) + ' cycles.\n')
    
if runOur_oneStage_random:
    print('\n=====For-Back Iter Random (With One Stage) Final Results=====\n')
    print('The optimal objective value (prob of miss detection) is ' + str(optObjVal_our_oneStage_random) + ' .\n')
    if ifPrintPlan:
        print('\nThe optimal search plan is:\n\n' + str(optActionVec_our_oneStage_random) + '\n')
    print('Time taken to run is %.2f' % (time_elapsed_our_oneStage_random) + ' .\n')
    print('Random repetitions = ' + str(randomRepetitionN_our_oneStage_random) + ' .\n')
    
if runOur_twoStage_random:
    print('\n=====For-Back Iter Random (With Two Stages) Final Results=====\n')
    print('The optimal objective value (prob of miss detection) is ' + str(optObjVal_our_twoStage_random) + ' .\n')
    if ifPrintPlan:
        print('\nThe optimal search plan is:\n\n' + str(optActionVec_our_twoStage_random) + '\n')
    print('Time taken to run is %.2f' % (time_elapsed_our_twoStage_random) + ' .\n')
    print('Random repetitions = ' + str(randomRepetitionN_our_twoStage_random) + ' .\n')
    
if runRoyset2:
    print('\n=====Cutting-Plane (Royset-Sato Alg 2) Final Results=====\n')
    print('The optimal objective value (prob of miss detection) is ' + str(optObjVal_royset2) + ' .\n')
    if ifPrintPlan:
        print('\nThe optimal search plan is:\n\n' + str(optActionVec_royset2) + '\n')
    print('Time taken to run is %.2f' % (time_elapsed_royset2) + ' .\n')
    
if runRoyset3:
    print('\n=====Cutting-Plane (Royset-Sato Alg 3) Final Results=====\n')
    print('The optimal objective value (prob of miss detection) is ' + str(optObjVal_royset3) + ' .\n')
    if ifPrintPlan:
        print('\nThe optimal search plan is:\n\n' + str(optActionVec_royset3) + '\n')
    print('Time taken to run is %.2f' % (time_elapsed_royset3) + ' .\n')
    
if runDp:
    print('\n=====POMDP Final Results=====\n')
    print('The optimal objective value (prob of miss detection) is ' + str(optObjVal_dp) + ' .\n')
    if ifPrintPlan:
        print('\nThe optimal search plan is:\n\n' + str(optActions_dp) + '\n')
    print('Time taken to run is %.2f' % (time_elapsed_dp) + ' .\n')
    
if runBrute:
    print('\n=====Brute Force Final Results=====\n')
    print('The optimal objective value (prob of miss detection) is ' + str(optObjVal_brute) + ' .\n')
    if ifPrintPlan:
        print('\nThe optimal search plan is:\n\n' + str(optActionVec_brute) + '\n')
    print('Time taken to run is %.2f' % (time_elapsed_brute) + ' .\n')